In [ ]:
import pandas as pd
import numpy as np
from tensorflow.keras.applications import InceptionResNetV2
from keras.models import Sequential
from keras.layers import Dense, GlobalAveragePooling2D
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.utils import shuffle

In [ ]:
# ! pip install -q kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets list

mkdir: cannot create directory ‘/root/.kaggle’: File exists
ref                                                            title                                                size  lastUpdated          downloadCount  voteCount  usabilityRating  
-------------------------------------------------------------  --------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
meirnizri/covid19-dataset                                      COVID-19 Dataset                                      5MB  2022-11-13 15:47:17           8302        248  1.0              
mattop/alcohol-consumption-per-capita-2016                     Alcohol Consumption Per Capita 2016                   4KB  2022-12-09 00:03:11            704         27  1.0              
thedevastator/jobs-dataset-from-glassdoor                      Salary Prediction                                     3MB  2022-11-16 13:52:31           5308        116  1.0              
swapt

In [ ]:
! kaggle datasets download -d ma7555/cat-breeds-dataset

100% 1.92G/1.93G [00:13<00:00, 177MB/s]
100% 1.93G/1.93G [00:13<00:00, 149MB/s]


In [ ]:
! mkdir cats
! unzip cat-breeds-dataset.zip -d cats

In [ ]:
array_paths = []
for dirpath, dirnames, filenames in os.walk('cats/images'):
  for filename in filenames:
    ids = filename.split('.')[0]
    array_paths.append({'breed': dirpath.split('/')[-1], 'path': os.path.join(dirpath, filename)})
file_paths = pd.DataFrame(array_paths)
file_paths

,breed,path
0,Ocicat,cats/images/Ocicat/21034865_503.jpg
1,Ocicat,cats/images/Ocicat/17994387_571.jpg
2,Ocicat,cats/images/Ocicat/21407332_484.jpg
3,Ocicat,cats/images/Ocicat/22681423_450.jpg
4,Ocicat,cats/images/Ocicat/20762910_507.jpg
...,...,...
126602,Somali,cats/images/Somali/20795997_90.jpg
126603,Somali,cats/images/Somali/25580528_70.jpg
126604,Somali,cats/images/Somali/44469436_11.jpg
126605,Somali,cats/images/Somali/32607051_45.jpg


In [ ]:
file_paths = (file_paths.sample(frac = 1).reset_index()).drop(columns = 'index')
file_paths

,breed,path
0,Domestic Medium Hair,cats/images/Domestic Medium Hair/46540127_1982...
1,Domestic Short Hair,cats/images/Domestic Short Hair/46666263_1079.jpg
2,Bombay,cats/images/Bombay/24019365_7143.jpg
3,Domestic Short Hair,cats/images/Domestic Short Hair/46194968_53330...
4,Tortoiseshell,cats/images/Tortoiseshell/43546304_6616.jpg
...,...,...
126602,Domestic Short Hair,cats/images/Domestic Short Hair/46319840_42459...
126603,Domestic Long Hair,cats/images/Domestic Long Hair/46542473_19548.jpg
126604,Domestic Short Hair,cats/images/Domestic Short Hair/46552422_18453...
126605,Applehead Siamese,cats/images/Applehead Siamese/14474662_643.jpg


In [ ]:
value_counts = file_paths['breed'].value_counts()
counts = {}
for i in range(value_counts.shape[0]):
  if value_counts.index[i].split(' ')[0] != 'Domestic' and value_counts.index[i].split(' ')[0] != 'Dilute':
    counts[value_counts.index[i]] = value_counts[i]
counts = pd.Series(counts)[:10]
counts

American Shorthair    5295
Persian               4018
Tortoiseshell         3963
Calico                3468
Torbie                3396
Tuxedo                3181
Tabby                 3012
Siamese               2888
Ragdoll               2669
Bengal                2477
dtype: int64

In [ ]:
list_samples = [None] * 10
for i in range(10):
  list_samples[i] = file_paths[file_paths['breed'] == counts.index[i]].sample(counts[-1])
balanced = pd.concat(list_samples)
balanced

,breed,path
4536,American Shorthair,cats/images/American Shorthair/46711880_114.jpg
6905,American Shorthair,cats/images/American Shorthair/46011283_1587.jpg
2877,American Shorthair,cats/images/American Shorthair/44932385_4485.jpg
4025,American Shorthair,cats/images/American Shorthair/43543627_6460.jpg
2595,American Shorthair,cats/images/American Shorthair/42288753_8528.jpg
...,...,...
119360,Bengal,cats/images/Bengal/30126238_5741.jpg
120348,Bengal,cats/images/Bengal/36053793_3762.jpg
120939,Bengal,cats/images/Bengal/35987311_3784.jpg
119486,Bengal,cats/images/Bengal/25393254_7303.jpg


In [ ]:
train, test = train_test_split(balanced, test_size=0.2)

In [ ]:
datagen = ImageDataGenerator(rescale=1./255, rotation_range=30, width_shift_range=0.2, height_shift_range=0.2, horizontal_flip=True)
x_train = datagen.flow_from_dataframe(train, x_col='path', y_col='breed', target_size=(300, 300))
x_test = datagen.flow_from_dataframe(test, x_col='path', y_col='breed', target_size=(300, 300))

Found 19816 validated image filenames belonging to 10 classes.
Found 4954 validated image filenames belonging to 10 classes.


In [ ]:
labels = x_train.class_indices
labels

{'American Shorthair': 0,
 'Bengal': 1,
 'Calico': 2,
 'Persian': 3,
 'Ragdoll': 4,
 'Siamese': 5,
 'Tabby': 6,
 'Torbie': 7,
 'Tortoiseshell': 8,
 'Tuxedo': 9}

In [ ]:
incep = InceptionResNetV2(include_top=False, weights='imagenet', input_shape=(300, 300, 3))
for layer in incep.layers:
  layer.trainable = False
incep.summary()

219055592/219055592 [==============================] - 10s 0us/step
Model: "inception_resnet_v2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 300, 300, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 149, 149, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 149, 149, 32  96         ['conv2d[0][0]']                 
 alization) 

In [ ]:
model = Sequential()
model.add(incep)
model.add(GlobalAveragePooling2D())
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(10, activation='softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_resnet_v2 (Functi  (None, 8, 8, 1536)       54336736  
 onal)                                                           
                                                                 
 global_average_pooling2d_1   (None, 1536)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_3 (Dense)             (None, 128)               196736    
                                                                 
 dense_4 (Dense)             (None, 64)                8256      
                                                                 
 dense_5 (Dense)             (None, 10)                650       
                                                                 
Total params: 54,542,378
Trainable params: 205,642
Non

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(x_train, validation_data=x_test, epochs=15, steps_per_epoch=100)

Epoch 1/15
100/100 [==============================] - 203s 2s/step - loss: 1.5934 - accuracy: 0.4134 - val_loss: 1.4339 - val_accuracy: 0.4849
Epoch 2/15
100/100 [==============================] - 188s 2s/step - loss: 1.2977 - accuracy: 0.5241 - val_loss: 1.2569 - val_accuracy: 0.5521
Epoch 3/15
100/100 [==============================] - 189s 2s/step - loss: 1.2491 - accuracy: 0.5516 - val_loss: 1.2105 - val_accuracy: 0.5650
Epoch 4/15
100/100 [==============================] - 190s 2s/step - loss: 1.2072 - accuracy: 0.5763 - val_loss: 1.2369 - val_accuracy: 0.5648
Epoch 5/15
100/100 [==============================] - 186s 2s/step - loss: 1.1734 - accuracy: 0.5847 - val_loss: 1.2007 - val_accuracy: 0.5563
Epoch 6/15
100/100 [==============================] - 189s 2s/step - loss: 1.1879 - accuracy: 0.5743 - val_loss: 1.1616 - val_accuracy: 0.5931
Epoch 7/15
100/100 [==============================] - 186s 2s/step - loss: 1.1307 - accuracy: 0.5963 - val_loss: 1.1379 - val_accuracy: 0.5971

In [ ]:
model.evaluate(x_test)[1]

155/155 [==============================] - 115s 738ms/step - loss: 1.0871 - accuracy: 0.6102


0.6102139949798584

In [ ]:
predictions = model.predict(x_test)

155/155 [==============================] - 117s 736ms/step


In [ ]:
predictions = np.argmax(predictions, axis=1)
predictions

array([4, 0, 8, ..., 5, 0, 7])

In [ ]:
test['breed'].replace(labels, inplace=True)

In [ ]:
confusion_matrix(test['breed'], predictions)

array([[37, 54, 40, 47, 24, 47, 35, 66, 54, 78],
       [36, 47, 49, 80, 39, 50, 40, 63, 50, 63],
       [43, 52, 46, 63, 43, 34, 36, 48, 58, 64],
       [46, 58, 26, 45, 39, 39, 38, 64, 50, 76],
       [28, 57, 38, 54, 46, 38, 43, 66, 52, 70],
       [54, 55, 35, 65, 50, 42, 22, 50, 53, 72],
       [40, 67, 39, 45, 45, 36, 40, 71, 54, 78],
       [36, 48, 52, 60, 34, 43, 38, 46, 48, 81],
       [26, 54, 32, 59, 46, 38, 31, 62, 66, 69],
       [36, 63, 57, 61, 49, 37, 44, 47, 51, 68]])

In [ ]:
model.save('model.h5')